# 🧪 Notebook 03: Val/Test Transforms — No Augmentation

**Purpose:** Build a deterministic transform pipeline for validation and test data (no random augmentations).

**What you'll learn:** Why evaluation needs consistency, and how to ensure reproducible metrics.


## 🎯 Concept Primer: Why NO Augmentation for Val/Test?

### Training vs Evaluation: Different Goals

**Training:**
- Goal: Learn robust features that generalize
- Strategy: Augment data to create synthetic variety
- Random transforms: Flip, rotate, color jitter

**Validation/Test:**
- Goal: Measure performance consistently
- Strategy: Apply only deterministic transforms
- No randomness: Same image → same output every time

### What Breaks Without This Rule?

Imagine if validation used random augmentations:

```python
# Run 1: Validation accuracy = 85%
# Run 2: Validation accuracy = 82%
# Run 3: Validation accuracy = 87%
```

**Problem:** You can't trust the metrics! Accuracy changes due to randomness, not model improvement.

### val_test_transform Pipeline

```
✅ CORRECT (Deterministic):
Resize(96,96) → ToTensor() → Normalize(mean=[0.5]*3, std=[0.5]*3)

❌ WRONG (Has randomness):
Resize → RandomHorizontalFlip → ToTensor → Normalize
```

**Key rule:** Normalization parameters **must match training** exactly.


## 📚 Learning Objectives

By the end of this notebook, you will:

1. ✅ Build `val_test_transform` with **only** deterministic transforms
2. ✅ Use the **same** normalization as training (mean=std=0.5)
3. ✅ Test transform on a sample image
4. ✅ Verify output shape: `[3, 96, 96]`
5. ✅ Understand why evaluation metrics require consistency


## ✅ Acceptance Criteria

Your val/test transform is correct when:

- [ ] `val_test_transform` is a `transforms.Compose` object
- [ ] It contains **only**: Resize, ToTensor, Normalize (no Random* transforms)
- [ ] Normalization uses `mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]` (matching training)
- [ ] Applying transform produces shape `[3, 96, 96]`
- [ ] Running transform twice on the **same image** produces **identical** tensors


---

## 💻 TODO 1: Import Required Libraries

**What you need:**
- `torchvision.transforms`
- `PIL.Image`
- `torch` (to test tensor equality)

**Expected behavior:** Imports run without errors.


In [1]:
# TODO 1: Import libraries
# Hint: from torchvision import transforms
# Hint: from PIL import Image
# Hint: import torch

# YOUR CODE HERE
from torchvision import transforms
from PIL import Image
import torch

print("✅ Imports successful")


✅ Imports successful


---

## 💻 TODO 2: Build val_test_transform (Deterministic Only!)

**What you need to compose (in this order):**

1. **`transforms.Resize((96, 96))`** — Ensure all images are 96×96
2. **`transforms.ToTensor()`** — Convert PIL image → tensor [0,1]
3. **`transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])`** — Normalize to [-1,1]

**❗ DO NOT INCLUDE:**
- ❌ RandomHorizontalFlip
- ❌ RandomRotation
- ❌ ColorJitter
- ❌ Any Random* transforms

**Expected output:** A `transforms.Compose` object stored in `val_test_transform`.


In [2]:
# TODO 2: Create val_test_transform (only deterministic transforms!)
# Hint: val_test_transform = transforms.Compose([...])
# Hint: Only include Resize, ToTensor, Normalize

# YOUR CODE HERE
val_test_transform = transforms.Compose([
    transforms.Resize((96, 96)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

print("✅ val_test_transform created:")
print(val_test_transform)


✅ val_test_transform created:
Compose(
    Resize(size=(96, 96), interpolation=bilinear, max_size=None, antialias=True)
    ToTensor()
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
)


---

## 💻 TODO 3: Verify Determinism — Apply Transform Twice

**What you need to do:**
1. Load a sample image from `../data/pcam_images/`
2. Apply `val_test_transform` **twice** to the same image
3. Check if the two tensors are **exactly equal** using `torch.equal()`

**Expected output:**
```
✅ Transform applied twice
   Tensor shapes match: True
   Tensors are identical: True  (Deterministic!)
```

If `torch.equal()` returns `False`, you accidentally included a random transform!


In [4]:
# TODO 3: Test determinism by applying transform twice
# Hint: sample_img = Image.open('...')
# Hint: transformed_1 = val_test_transform(sample_img)
# Hint: transformed_2 = val_test_transform(sample_img)
# Hint: torch.equal(transformed_1, transformed_2)

import os
import torch

sample_images = os.listdir('../data/pcam_images/')
sample_path = os.path.join('../data/pcam_images/', sample_images[0])

# YOUR CODE HERE
# Load image (we need to reload it for a fresh copy each time)
sample_img_1 = Image.open(sample_path)
sample_img_2 = Image.open(sample_path)

# Apply transform to both copies
transformed_1 = val_test_transform(sample_img_1)
transformed_2 = val_test_transform(sample_img_2)

# Check equality
are_equal = torch.equal(transformed_1, transformed_2)

print("✅ Transform applied twice")
print(f"   Shape: {transformed_1.shape}")
print(f"   Results are identical: {are_equal}")


✅ Transform applied twice
   Shape: torch.Size([3, 96, 96])
   Results are identical: True


---

## 🤔 Reflection Prompts

### Question 1: What Breaks If Val/Test Had Random Augmentation?

Imagine you're training a model and checking validation accuracy every epoch:

**Scenario A: val_test_transform includes RandomHorizontalFlip**

| Epoch | Val Accuracy |
|-------|--------------|
| 1 | 78% |
| 2 | 82% |
| 3 | 79% |
| 4 | 85% |
| 5 | 80% |

**Questions:**
- Did the model improve from Epoch 2 → 3?
- Can you trust Epoch 4's 85% to select the best model?
- What causes the fluctuations?

**Your analysis:**
> If validation/test had random augmentation, the results would be completely randomized and impossible to compare. From a computational point of view, you cannot validate or test because the model won't find consistent patterns. We need deterministic answers to check correctly. Unless you have a quantum computer, you cannot handle this level of randomness. What matters most is that it is deterministic - there should be nothing dependent on randomness, otherwise it's impossible to find meaningful results.

---

### Question 2: Why Must Normalization Match Training?

Consider this **incorrect** setup:
```python
# Training
train_transform = Normalize(mean=[0.5]*3, std=[0.5]*3)  # [-1,1]

# Val/Test (WRONG!)
val_test_transform = Normalize(mean=[0.485, 0.456, 0.406], 
                                std=[0.229, 0.224, 0.225])  # ImageNet stats
```

**Question:** What happens when the model trained on [-1,1] data receives ImageNet-normalized inputs?

**Your explanation:**
> If the image is not normalized to the same scale, you'll have mismatched data distributions. One image might be completely black, another completely white, or whatever that means. If we normalize to different scales with different values, there's a mismatch in what should be the benchmarks. The normalization should be consistent across train/val/test - same scale, not different scales. This ensures the model receives data in the same format it was trained on.

---

### Question 3: Deterministic vs Reproducible

**Deterministic:** Same input → same output (no randomness)
**Reproducible:** Same code → same result (controlled random seed)

Which matters more for validation/test transforms? Why?

**Your answer:**
> Deterministic matters more for validation/test transforms. We need to ensure that the same input always produces the same output, with no randomness involved. This allows for consistent evaluation and fair comparison of model performance across different runs.

---


## 🚀 Next Steps

Perfect! You've built a deterministic transform for evaluation.

**Move to Notebook 04:** Load Val/Test DataLoaders

**Key Takeaway:** Evaluation = Deterministic + No Shuffle!
